In [1]:
from models import basic_dnn
from scripts.SEMIC import *
from sklearn import metrics
import tensorflow as tf

In [2]:
ckpt_path = "./out/SEMIC/"
phase_list = ['train', 'valid', 'test']
base_dir = '/media/cocoan-gpu1/wani8T/data/SEMIC_npy/'

In [3]:
semic = load_dataset(shuffle=True, dataset_path= base_dir, batch_size = 20, subj_group_size=1, nan_to_zero=True)

Train dataset Subject list : 001, 010, 020, 030, 040, 050, 002, 021, 051, 041, 031, 012, 022, 032, 003, 052, 042, 013, 004, 023, 033, 053, 043, 014, 024, 034, 006, 015, 045, 035, 055, 016, 007, 026, 046, 056, 008, 027, 037, 047, 057, 009, 028, 038, 048, 058, 018, 029, 049, 039, 059, 019

Test dataset Subject list : 036, 011, 054, 005, 044, 025



In [4]:
net = basic_dnn.create((79*95*79),2)

In [5]:
num_epochs = 10000
lowest_loss=None
highest_auc=None

In [ ]:
with net.sess as sess:
    with tf.device('/gpu:0'):
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        
        summary_writer = tf.summary.FileWriter(ckpt_path, net.sess.graph)

        for epoch in range(num_epochs):

            feed_dict = {phase : {} for phase in phase_list}

            loss = {phase : 0 for phase in phase_list}
            count = {phase : 0 for phase in phase_list}
            metrics = {phase : {} for phase in ['auc', 'sensitivity', 'specitivity']}

            for method in metrics.keys():
                metrics[method] = {phase : 0 for phase in phase_list}

            for phase in phase_list:
                prediction = []
                answer = []    

                for _ in range(semic.num_subj_group[phase]):
                    if phase != 'valid' : semic.load(phase)
                        
                    for i in range(len(semic.batchset[phase]['labels'])):
                        semic.batchset[phase]['fmri'][i] = np.reshape(semic.batchset[phase]['fmri'][i], (-1, 79*95*79))
                        
                        feed_dict = {net.x: semic.batchset[phase]['fmri'][i],
                                     net.y: semic.batchset[phase]['labels'][i],
                                     net.keep_prob: 0.7 if phase=='train' else 1.0,
                                     net.is_train : True if phase=='train' else False}
                    
                        if phase == 'train' :
                            feed_dict.update({net.lr : 1. })
                            pred, cost, _ = sess.run([net.output, net.loss, net.train_op], feed_dict=feed_dict)
                        else :
                            pred, cost = sess.run([net.output, net.loss], feed_dict=feed_dict)

                        prediction.append(pred.argmax(-1))
                        answer.append(semic.batchset[phase]['labels'][i])
                        loss[phase] += cost
                        count[phase] += 1
                        
                        print("Loss : {}".format(loss[phase]))

                prediction = np.concatenate(prediction, axis=0)
                answer = np.concatenate(answer, axis=0)

                fpr, tpr, _ = metrics.roc_curve(answer, prediction, pos_label=1)
                tn, fp, fn, tp = metrics.confusion_matrix(answer, prediction).ravel()
                metrics['auc'][phase] = metrics.auc(fpr, tpr)
                metrics['sensitivity'][phase] = tp / (tp+fn)
                metrics['specitivity'][phase] = tn / (tn+fp)
                
                print("AUC : {}".format(metrics.auc(fpr, tpr)))
                
                summ = tf.Summary()

                for phase in phase_list:
                    summ.value.add(tag=phase+'_loss', 
                                   simple_value=loss[phase]/count[phase] if count[phase] !=0 else 0)
                    for method in metrics.keys():
                        summ.value.add(tag=phase+'_'+method, 
                                       simple_value=metrics[method][phase] if count[phase] !=0 else 0)

                summary_writer.add_summary(summ,epoch)

                if epoch > 0 and phase == 'valid':
                    if lowest_loss == None or lowest_loss > loss[phase]/count[phase] :
                        lowest_loss = loss[phase]/count[phase]
                        saver.save(sess, ckpt_path+"basic_dnn_lowest_loss.ckpt")
                    if highest_auc == None or highest_auc < metrics['auc'][phase] :
                        highest_auc = metrics['auc'][phase]
                        saver.save(sess, ckpt_path+"basic_dnn_highest_auc.ckpt")
    

Loss : 1.2555046081542969
Loss : 45.725833892822266
Loss : 51.04031467437744
Loss : 76.19288158416748
Loss : 82.2156400680542
Loss : 84.93522429466248
Loss : 89.40159010887146
Loss : 95.3393862247467
Loss : 97.99100995063782
Loss : 105.15016674995422
Loss : 107.16719222068787
Loss : 109.02951669692993
Loss : 111.57619452476501
Loss : 112.8505859375
Loss : 113.66370666027069
Loss : 114.36336636543274
Loss : 114.77736201882362
Loss : 115.09079349040985
Loss : 115.6727603673935
Loss : 116.34782814979553
Loss : 117.14308476448059
Loss : 117.89820039272308
Loss : 118.19832241535187
Loss : 120.3492761850357
Loss : 120.99375540018082
Loss : 121.975330889225
Loss : 122.62135833501816
Loss : 123.06660664081573
Loss : 124.38209962844849
Loss : 124.6604175567627
Loss : 124.78689680993557
Loss : 125.0460392087698
Loss : 125.38632093369961
Loss : 128.16559921205044
Loss : 128.37121744453907
Loss : 132.5261518508196
Loss : 132.79217948019505
Loss : 133.17599983513355
Loss : 133.64332105219364
Loss :

Loss : 406.1994073987007
Loss : 406.89271610975266
Loss : 407.5879974961281
Loss : 408.28437799215317
Loss : 408.9793271422386
Loss : 409.6726054549217
Loss : 410.3662096261978
Loss : 411.06110912561417
Loss : 411.7560616135597
Loss : 412.4498298764229
Loss : 413.14297956228256
Loss : 413.8367420434952
Loss : 414.5311201810837
Loss : 415.22512394189835
Loss : 415.9184014201164
Loss : 416.6116409897804
Loss : 417.30538392066956
Loss : 417.99926978349686
Loss : 418.6927369236946
Loss : 419.38588708639145
Loss : 420.0792187452316
Loss : 420.77282679080963
Loss : 421.4663425683975
Loss : 422.159566283226
Loss : 422.8527337908745
Loss : 423.54609060287476
Loss : 424.2395302057266
Loss : 424.93281972408295
Loss : 425.62597090005875
Loss : 426.3191793560982
Loss : 427.01250153779984
Loss : 427.70579820871353
Loss : 428.3989802002907
Loss : 429.09213334321976
Loss : 429.78535908460617
Loss : 430.47862070798874
Loss : 431.1718251109123
Loss : 431.86497408151627
Loss : 432.5581448674202
Loss : 4

Loss : 642.1542276740074
Loss : 642.847374856472
Loss : 643.5405220389366
Loss : 644.2336692214012
Loss : 644.9268164038658
Loss : 645.6199635863304
Loss : 646.313110768795
Loss : 647.0062579512596
Loss : 647.6994051337242
Loss : 648.3925523161888
Loss : 649.0856994986534
Loss : 649.778846681118
Loss : 650.4719938635826
Loss : 651.1651410460472
Loss : 651.8582882285118
Loss : 652.5514354109764
Loss : 653.244582593441
Loss : 653.9377297759056
Loss : 654.6308769583702
Loss : 655.3240241408348
Loss : 656.0171713232994
Loss : 656.710318505764
Loss : 657.4034656882286
Loss : 658.0966128110886
Loss : 658.7897599935532
Loss : 659.4829071760178
Loss : 660.1760543584824
Loss : 660.869201540947
Loss : 661.5623487234116
Loss : 662.2554959058762
Loss : 662.9486430883408
Loss : 663.6417902708054
Loss : 664.33493745327
Loss : 665.0280846357346
Loss : 665.7212318181992
Loss : 666.4143790006638
Loss : 667.1075261831284
Loss : 667.800673365593
Loss : 668.4938205480576
Loss : 669.1869677305222
Loss : 66

Loss : 872.2792835235596
Loss : 872.9724307060242
Loss : 873.6655778884888
Loss : 874.3587250709534
Loss : 875.051872253418
Loss : 875.7450194358826
Loss : 876.4381666183472
Loss : 877.1313138008118
Loss : 877.8244609832764
Loss : 878.517608165741
Loss : 879.2107553482056
Loss : 879.9039025306702
Loss : 880.5970497131348
Loss : 881.2901968955994
Loss : 881.983344078064
Loss : 882.6764912605286
Loss : 883.3696384429932
Loss : 884.0627856254578
Loss : 884.7559328079224
Loss : 885.449079990387
Loss : 886.1422271728516
Loss : 886.8353743553162
Loss : 887.5285215377808
Loss : 888.22166877985
Loss : 888.9148159623146
Loss : 889.6079631447792
Loss : 890.3011103272438
Loss : 890.9942575097084
Loss : 891.687404692173
Loss : 892.3805518746376
Loss : 893.0736990571022
Loss : 893.7668462395668
Loss : 894.4599934220314
Loss : 895.153140604496
Loss : 895.8462877869606
Loss : 896.5394349694252
Loss : 897.2325821518898
Loss : 897.9257293343544
Loss : 898.6188764572144
Loss : 899.312023639679
Loss : 90

Loss : 1101.1057332158089
Loss : 1101.7988803982735
Loss : 1102.492027580738
Loss : 1103.1851747632027
Loss : 1103.8783219456673
Loss : 1104.5714690685272
Loss : 1105.2646162509918
Loss : 1105.9577634334564
Loss : 1106.650910615921
Loss : 1107.3440577983856
Loss : 1108.0372049212456
Loss : 1108.7303519845009
Loss : 1109.4234991669655
Loss : 1110.11664634943
Loss : 1110.8097935318947
Loss : 1111.5029407143593
Loss : 1112.1960878968239
Loss : 1112.8892350792885
Loss : 1113.582382261753
Loss : 1114.275529384613
Loss : 1114.9686765670776
Loss : 1115.6618237495422
Loss : 1116.3549709320068
Loss : 1117.0481181144714
Loss : 1117.7412652373314
Loss : 1118.434412419796
Loss : 1119.127559542656
Loss : 1119.8207067251205
Loss : 1120.5138539075851
Loss : 1121.2070010900497
Loss : 1121.9001482725143
Loss : 1122.593295454979
Loss : 1123.2864426374435
Loss : 1123.9795898199081
Loss : 1124.6727368831635
Loss : 1125.365884065628
Loss : 1126.0590312480927
Loss : 1126.7521784305573
Loss : 1127.4453256130

...

Loss : 5426.104797899723
Loss : 5426.797945082188
Loss : 5427.491092264652
Loss : 5428.184239447117
Loss : 5428.877386629581
Loss : 5429.570533812046
Loss : 5430.263680994511
Loss : 5430.956828117371
Loss : 5431.649975299835
Loss : 5432.3431224823
Loss : 5433.036269664764
Loss : 5433.729416847229
Loss : 5434.422564029694
Loss : 5435.115711212158
Loss : 5435.808858394623
Loss : 5436.502005577087
Loss : 5437.195152819157
Loss : 5437.888300001621
Loss : 5438.581447184086
Loss : 5439.274594306946
Loss : 5439.96774148941
Loss : 5440.660888671875
Loss : 5441.35403585434
Loss : 5442.047183036804
Loss : 5442.740330219269
Loss : 5443.433477401733
Loss : 5444.126624584198
Loss : 5444.819771766663
Loss : 5445.512918949127
Loss : 5446.206066250801
Loss : 5446.899213433266
Loss : 5447.59236061573
Loss : 5448.285507798195
Loss : 5448.9786549806595
Loss : 5449.671802163124
Loss : 5450.364949345589
Loss : 5451.058096528053
Loss : 5451.751243829727
Loss : 5452.444391131401
Loss : 5453.137538433075
Loss

Loss : 5657.090532064438
Loss : 5657.78371411562
Loss : 5658.476863384247
Loss : 5659.1700702905655
Loss : 5659.863351762295
Loss : 5660.556652367115
Loss : 5661.249904751778
Loss : 5661.943087875843
Loss : 5662.636235654354
Loss : 5663.329399347305
Loss : 5664.022603213787
Loss : 5664.715831100941
Loss : 5665.409047245979
Loss : 5666.102228999138
Loss : 5666.795382022858
Loss : 5667.488530635834
Loss : 5668.181695818901
Loss : 5668.8748797774315
Loss : 5669.568068146706
Loss : 5670.261244535446
Loss : 5670.954403042793
Loss : 5671.647550940514
Loss : 5672.340700745583
Loss : 5673.033859968185
Loss : 5673.727027177811
Loss : 5674.420194447041
Loss : 5675.113354623318
Loss : 5675.806506216526
Loss : 5676.4996535182
Loss : 5677.192802250385
Loss : 5677.885955750942
Loss : 5678.579112946987
Loss : 5679.272270083427
Loss : 5679.965423762798
Loss : 5680.658573329449
Loss : 5681.351720631123
Loss : 5682.044868409634
Loss : 5682.738018333912
Loss : 5683.431170165539
Loss : 5684.124322295189
L

Loss : 5887.310438156128
Loss : 5888.0035853385925
Loss : 5888.696732521057
Loss : 5889.389879703522
Loss : 5890.083026885986
Loss : 5890.776174068451
Loss : 5891.4693212509155
Loss : 5892.16246843338
Loss : 5892.855615615845
Loss : 5893.548762798309
Loss : 5894.241909980774
Loss : 5894.935057044029
Loss : 5895.628204226494
Loss : 5896.321351408958
Loss : 5897.014498591423
Loss : 5897.707645773888
Loss : 5898.400792956352
Loss : 5899.093940138817
Loss : 5899.787087202072
Loss : 5900.480234503746
Loss : 5901.173381686211
Loss : 5901.866528868675
Loss : 5902.55967605114
Loss : 5903.252823114395
Loss : 5903.94597029686
Loss : 5904.639117479324
Loss : 5905.332264661789
Loss : 5906.025411963463
Loss : 5906.718559145927
Loss : 5907.411706328392
Loss : 5908.104853510857
Loss : 5908.798000693321
Loss : 5909.491147875786
Loss : 5910.18429505825
Loss : 5910.877442240715
Loss : 5911.57058942318
Loss : 5912.26373654604
Loss : 5912.956883728504
Loss : 5913.650030910969
Loss : 5914.343178093433
Loss

Loss : 6117.525410950184
Loss : 6118.218560695648
Loss : 6118.911711990833
Loss : 6119.6048656105995
Loss : 6120.29802274704
Loss : 6120.991185843945
Loss : 6121.684358537197
Loss : 6122.377546727657
Loss : 6123.07076048851
Loss : 6123.764016211033
Loss : 6124.4573412537575
Loss : 6125.150781095028
Loss : 6125.844411313534
Loss : 6126.538355410099
Loss : 6127.232811927795
Loss : 6127.928085684776
Loss : 6128.624607801437
Loss : 6129.3228931427
Loss : 6130.02333676815
Loss : 6130.725762665272
Loss : 6131.4289072752
Loss : 6132.130533337593
Loss : 6132.828732252121
Loss : 6133.523469686508
Loss : 6134.216638445854
Loss : 6134.910565435886
Loss : 6135.606434702873
Loss : 6136.303671717644
Loss : 6137.0005621910095
Loss : 6137.695735454559
Loss : 6138.389285445213
Loss : 6139.082481980324
Loss : 6139.776517689228
Loss : 6140.471546888351
Loss : 6141.166739165783
Loss : 6141.861164391041
Loss : 6142.554646193981
Loss : 6143.247797310352
Loss : 6143.9413350224495
Loss : 6144.6354342103
Loss 

Loss : 6347.123062610626
Loss : 6347.816211104393
Loss : 6348.5093594789505
Loss : 6349.20250749588
Loss : 6349.895655035973
Loss : 6350.588802278042
Loss : 6351.281949460506
Loss : 6351.97509676218
Loss : 6352.668244421482
Loss : 6353.361392199993
Loss : 6354.054539978504
Loss : 6354.747687518597
Loss : 6355.44083493948
Loss : 6356.133982121944
Loss : 6356.827129304409
Loss : 6357.520276486874
Loss : 6358.2134237885475
Loss : 6358.906571209431
Loss : 6359.599718630314
Loss : 6360.292865991592
Loss : 6360.986013293266
Loss : 6361.679160475731
Loss : 6362.372307777405
Loss : 6363.065454959869
Loss : 6363.758602142334
Loss : 6364.451749444008
Loss : 6365.144896745682
Loss : 6365.838044047356
Loss : 6366.53119122982
Loss : 6367.2243384718895
Loss : 6367.917485654354
Loss : 6368.610632896423
Loss : 6369.303780078888
Loss : 6369.9969272613525
Loss : 6370.690074503422
Loss : 6371.383221685886
Loss : 6372.076368868351
Loss : 6372.769516050816
Loss : 6373.46266323328
Loss : 6374.155810415745
L